In [1]:
path = '/mnt/t3nfs01/data01/shome/micheli/CMSSW_HHbbgg/CMSSW_8_0_26/src/diphotons/Utils/macro/outPlots'
path ='/mnt/t3nfs01/data01/shome/nchernya/local/'
path = '/afs/cern.ch/work/n/nchernya/ETH/CMSSW_8_0_28/src/diphotons/Utils/macro/outPlots/'

In [2]:
import glob

In [3]:
#files = glob.glob('%s/out_20171130_17*node*_cat[0-9]*_mjj/cat_opt.json' % path)
#files = glob.glob('%s/out_20180420_170901_node_SM_cat[0-9]*_mjj/cat_opt.json' % path)
#date="20180628_163555"
#date = '20180604_153820'
date="20181001_163444"

files = glob.glob('%s/out_%s_node*_cat*_transformed_mjj/cat_opt.json' %(path,date))

In [4]:
files

['/afs/cern.ch/work/n/nchernya/ETH/CMSSW_8_0_28/src/diphotons/Utils/macro/outPlots/out_20181001_163444_node_SM_cat0_transformed_mjj/cat_opt.json',
 '/afs/cern.ch/work/n/nchernya/ETH/CMSSW_8_0_28/src/diphotons/Utils/macro/outPlots/out_20181001_163444_node_SM_cat1_transformed_mjj/cat_opt.json',
 '/afs/cern.ch/work/n/nchernya/ETH/CMSSW_8_0_28/src/diphotons/Utils/macro/outPlots/out_20181001_163444_node_SM_cat2_transformed_mjj/cat_opt.json',
 '/afs/cern.ch/work/n/nchernya/ETH/CMSSW_8_0_28/src/diphotons/Utils/macro/outPlots/out_20181001_163444_node_SM_cat3_transformed_mjj/cat_opt.json',
 '/afs/cern.ch/work/n/nchernya/ETH/CMSSW_8_0_28/src/diphotons/Utils/macro/outPlots/out_20181001_163444_node_SM_cat4_transformed_mjj/cat_opt.json',
 '/afs/cern.ch/work/n/nchernya/ETH/CMSSW_8_0_28/src/diphotons/Utils/macro/outPlots/out_20181001_163444_node_SM_cat5_transformed_mjj/cat_opt.json',
 '/afs/cern.ch/work/n/nchernya/ETH/CMSSW_8_0_28/src/diphotons/Utils/macro/outPlots/out_20181001_163444_node_SM_cat6_tr

In [5]:
import json 
import numpy as np

summary = { x:[] for x in ['node', 'cat', 'boundaries','ncat','fom','selections']  }

for fil in files:
    toks = fil.split('/')[-2].split('node_')
    name = toks[1].split('_')[0]
    cat = int(toks[1].split('_')[1].replace('cat',''))
    if name == 'SM': name = '0'
    node = int(name)
    with open(fil) as fin:
        info = json.loads(fin.read())
        fin.close()
    for key,val in info.items():
        val['node'] = node
        val['cat'] = cat
        val['boundaries'] = np.array(val['boundaries'])
        val['selections'] = np.array(val['selections']).astype(float)
        for col in summary.keys():
            summary[col].append( val[col] )

# print(summary)
    

In [6]:
import pandas as pd
df = pd.DataFrame(summary)

In [7]:
# df.set_index(['ncat','node'],drop=False,inplace=True# )

In [8]:
df.sort_index(inplace=True)
df

,boundaries,cat,fom,ncat,node,selections
0,"[1.01, 0.0]",0,-0.395,1,0,"[-25.0, -28.984]"
1,"[1.01, 0.0]",1,-2.259,1,0,"[-18.262, -27.522]"
2,"[1.01, 0.0]",2,-1.904,1,0,"[-25.308, -38.572]"
3,"[1.01, 0.0]",3,-2.176,1,0,"[-25.0, -26.56]"
4,"[1.01, 0.0]",4,-0.357,1,0,"[-20.31, -38.111]"
5,"[1.01, 0.0]",5,-1.676,1,0,"[-24.938, -38.113]"
6,"[1.01, 0.0]",6,-0.991,1,0,"[-24.988, -41.418]"
7,"[1.01, 0.0]",7,-0.817,1,0,"[-24.567, -37.531]"
8,"[1.01, 0.0]",8,-0.220,1,0,"[-26.56, -44.999]"
9,"[1.01, 0.0]",9,-0.700,1,0,"[-20.998, -34.125]"


In [9]:
df = df.query('fom<0')
df

,boundaries,cat,fom,ncat,node,selections
0,"[1.01, 0.0]",0,-0.395,1,0,"[-25.0, -28.984]"
1,"[1.01, 0.0]",1,-2.259,1,0,"[-18.262, -27.522]"
2,"[1.01, 0.0]",2,-1.904,1,0,"[-25.308, -38.572]"
3,"[1.01, 0.0]",3,-2.176,1,0,"[-25.0, -26.56]"
4,"[1.01, 0.0]",4,-0.357,1,0,"[-20.31, -38.111]"
5,"[1.01, 0.0]",5,-1.676,1,0,"[-24.938, -38.113]"
6,"[1.01, 0.0]",6,-0.991,1,0,"[-24.988, -41.418]"
7,"[1.01, 0.0]",7,-0.817,1,0,"[-24.567, -37.531]"
8,"[1.01, 0.0]",8,-0.220,1,0,"[-26.56, -44.999]"
9,"[1.01, 0.0]",9,-0.700,1,0,"[-20.998, -34.125]"


In [10]:
from sklearn.cluster import KMeans

In [11]:
boundaries = []

# for ncats in [1]:
for row in df['selections']:
    boundaries.append( row )

In [12]:
boundaries = np.array(boundaries)

df['min_mjj'] = boundaries[:,1].ravel().astype(np.float)
df['max_mjj'] = -boundaries[:,0].ravel().astype(np.float)
#boundaries[:,:,1].shape

In [13]:
#df = df.query('min_mjj > -50')
#df = df.query('max_mjj < 50')
#df

In [14]:
df['fom'] = -df['fom']

In [15]:
grouped = df.groupby('cat')

In [16]:
df[['node','max_mjj','min_mjj','fom']]

,node,max_mjj,min_mjj,fom
0,0,25.000,-28.984,0.395
1,0,18.262,-27.522,2.259
2,0,25.308,-38.572,1.904
3,0,25.000,-26.560,2.176
4,0,20.310,-38.111,0.357
5,0,24.938,-38.113,1.676
6,0,24.988,-41.418,0.991
7,0,24.567,-37.531,0.817
8,0,26.560,-44.999,0.220
9,0,20.998,-34.125,0.700


In [17]:
len(grouped)

12

In [23]:
tag='deepCSV'
pathsave = '/afs/cern.ch/work/n/nchernya/ETH/CMSSW_8_0_28/src/diphotons/Utils/macro/outPlots/plots/%s_%s/'%(date,tag)

In [25]:
import matplotlib.pyplot as plt

avgs = []

for group in grouped.groups:
    plt.clf()
    grp = grouped.get_group(group)
    min_fom = np.percentile(grp['fom'],20.)
    grp = grp[ grp['fom'] > min_fom ]
    ax = plt.subplot(2,1,1)
    plt.title('cat %d' % group)
    grp.plot.scatter( 'node', 'max_mjj', c='fom', ax=ax)    
    max_avg = (grp['max_mjj'] * grp['fom']).sum() / grp['fom'].sum()
    ax.plot((0,13),(max_avg,max_avg),'k-')
  #  ax.set_ylim(10,50)
    
    ax=plt.subplot(2,1,2)
    grp.plot.scatter( 'node', 'min_mjj', c='fom', ax=ax)
    min_avg = (grp['min_mjj'] * grp['fom']).sum() / grp['fom'].sum()
    ax.plot((0,13),(min_avg,min_avg),'k-')

    avgs.append([group,min_avg,max_avg])
    plt.savefig(pathsave+"clusteringMjj_%s_%s_cat%d"%(tag,date,group)+".pdf")

    plt.show()

ValueError: scatter requires x column to be numeric

In [18]:
avgs

NameError: name 'avgs' is not defined

In [32]:
avgs

array([], dtype=float64)

In [26]:
avgs = np.array(avgs)
round_to = 1.

In [27]:
rounded = np.round(avgs[:,1:3] / round_to)*round_to

IndexError: too many indices for array

In [28]:
rounded[:,1] = np.vstack( [rounded[:,1],np.ones_like(rounded[:,1])*40] ).min(axis=0)

NameError: name 'rounded' is not defined

In [29]:
rounded[0,1]

NameError: name 'rounded' is not defined

In [30]:
rounded+=125
rounded

NameError: name 'rounded' is not defined

In [36]:
###for 1 node only
#bound = np.round(np.vstack((df['min_mjj']+125,df['max_mjj']+125)),0)
#bound
bound = np.round(np.vstack((df['min_mjj'],df['max_mjj'])),0)
for i in range(len(bound)):
    for j in range(len(bound[i])):
        if bound[i][j]>30 : bound[i][j]=30
        if bound[i][j]<-30: bound[i][j]=-30
bound
final_mjj = np.round(np.vstack((bound[0]+125,bound[1]+125)),0)

########### For new code in flashgg :  cat0,cat1.... -> from here : cat3,2,1,0,7,6,5,4,11,10,9,8

print final_mjj

numMX_cat = 4
numMVA_cat = 3

cat = 3
going=1
flashgg_mjj = [[],[]]
num = 0
while going:
    flashgg_mjj[0].append(final_mjj[0][cat])
    flashgg_mjj[1].append(final_mjj[1][cat])
    num+=1
    if (cat==8) : going = 0
    if (cat%numMX_cat)==0 : cat+=2*numMX_cat-1
    else : cat-=1    

print flashgg_mjj


info_file = open(pathsave+"info_Mjj_%s_%s.txt"%(tag,date),"w") 
info_file.write(",".join([str(flashgg_mjj[0][i]) for i in range(0,len(flashgg_mjj[0]))]))
info_file.write("\n")
info_file.write(",".join([str(flashgg_mjj[1][i]) for i in range(0,len(flashgg_mjj[1]))]))
info_file.close()

[[  96.   97.   95.   98.   95.   95.   95.   95.   95.   95.   95.   95.]
 [ 150.  143.  150.  150.  145.  150.  150.  150.  152.  146.  142.  155.]]
[[98.0, 95.0, 97.0, 96.0, 95.0, 95.0, 95.0, 95.0, 95.0, 95.0, 95.0, 95.0], [150.0, 150.0, 143.0, 150.0, 150.0, 150.0, 150.0, 145.0, 155.0, 142.0, 146.0, 152.0]]


In [26]:
info_file = open(pathsave+"info_Mjj_%s_%s.txt"%(tag,date),"w") 
info_file.write(",".join([str(rounded[i][0]) for i in range(0,len(rounded))]))
info_file.write("\n")
info_file.write(",".join([str(rounded[i][1]) for i in range(0,len(rounded))]))
info_file.close()